# 🔌 Test des APIs - Job Market

Ce notebook teste la connexion aux 3 APIs principales du projet :
1. **The Muse** - Offres internationales
2. **Adzuna** - Offres France + données salariales
3. **France Travail** - Offres officielles France

**Objectifs :**
- Vérifier que les clés API fonctionnent
- Collecter 10 offres de test de chaque source
- Explorer la structure des données retournées
- Identifier les champs utiles pour notre projet

---

## 📦 Installation des dépendances

Si vous êtes sur **Google Colab**, exécutez cette cellule :

In [ ]:
# Décommenter si sur Google Colab
# !pip install requests python-dotenv pandas

## 🔑 Configuration des clés API

**Option A : Depuis un fichier .env (local)**
```python
from dotenv import load_dotenv
load_dotenv('../config/.env')
```

**Option B : Directement dans le code (Colab)**
```python
import os
os.environ['THEMUSE_API_KEY'] = 'votre_cle_ici'
```

In [ ]:
import requests
import json
import os
from datetime import datetime
import pandas as pd

# Option A : Charger depuis .env (si local)
# from dotenv import load_dotenv
# load_dotenv('../config/.env')

# Option B : Définir directement (si Colab)
# Remplacez par vos vraies clés après les avoir obtenues
THEMUSE_API_KEY = os.getenv('THEMUSE_API_KEY', 'YOUR_KEY_HERE')
ADZUNA_APP_ID = os.getenv('ADZUNA_APP_ID', 'YOUR_APP_ID')
ADZUNA_APP_KEY = os.getenv('ADZUNA_APP_KEY', 'YOUR_APP_KEY')
FRANCETRAVAIL_CLIENT_ID = os.getenv('FRANCETRAVAIL_CLIENT_ID', 'YOUR_CLIENT_ID')
FRANCETRAVAIL_CLIENT_SECRET = os.getenv('FRANCETRAVAIL_CLIENT_SECRET', 'YOUR_CLIENT_SECRET')

print("✅ Configuration chargée")

---
## 1️⃣ Test de The Muse API

**Documentation :** https://www.themuse.com/developers/api/v2

**Endpoint :** `GET https://www.themuse.com/api/public/jobs`

In [ ]:
def test_themuse():
    """Teste la connexion à The Muse API et récupère 10 offres"""
    
    print("🔍 Test de The Muse API...")
    
    url = "https://www.themuse.com/api/public/jobs"
    params = {
        'api_key': THEMUSE_API_KEY,
        'page': 1,
        'descending': True
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        jobs = data.get('results', [])
        
        print(f"✅ Connexion réussie !")
        print(f"📊 Nombre d'offres récupérées : {len(jobs)}")
        print(f"📄 Total disponible : {data.get('page_count', 'N/A')} pages")
        
        # Afficher un exemple
        if jobs:
            print("\n📝 Exemple d'offre :")
            job = jobs[0]
            print(f"  - Titre : {job.get('name', 'N/A')}")
            print(f"  - Entreprise : {job.get('company', {}).get('name', 'N/A')}")
            print(f"  - Localisation : {', '.join([loc.get('name', '') for loc in job.get('locations', [])])}")
            print(f"  - Niveau : {', '.join(job.get('levels', []))}")
            
        return jobs
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur : {e}")
        return []

# Exécuter le test
themuse_jobs = test_themuse()

---
## 2️⃣ Test de Adzuna API

**Documentation :** https://developer.adzuna.com/

**Endpoint :** `GET https://api.adzuna.com/v1/api/jobs/fr/search/1`

In [ ]:
def test_adzuna():
    """Teste la connexion à Adzuna API et récupère 10 offres françaises"""
    
    print("🔍 Test de Adzuna API...")
    
    url = f"https://api.adzuna.com/v1/api/jobs/fr/search/1"
    params = {
        'app_id': ADZUNA_APP_ID,
        'app_key': ADZUNA_APP_KEY,
        'results_per_page': 10,
        'what': 'data',  # Recherche exemple
        'content-type': 'application/json'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        jobs = data.get('results', [])
        
        print(f"✅ Connexion réussie !")
        print(f"📊 Nombre d'offres récupérées : {len(jobs)}")
        print(f"📄 Total disponible : {data.get('count', 'N/A')} offres")
        
        # Afficher un exemple
        if jobs:
            print("\n📝 Exemple d'offre :")
            job = jobs[0]
            print(f"  - Titre : {job.get('title', 'N/A')}")
            print(f"  - Entreprise : {job.get('company', {}).get('display_name', 'N/A')}")
            print(f"  - Localisation : {job.get('location', {}).get('display_name', 'N/A')}")
            print(f"  - Salaire : {job.get('salary_min', 'N/A')} - {job.get('salary_max', 'N/A')} €")
            
        return jobs
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur : {e}")
        return []

# Exécuter le test
adzuna_jobs = test_adzuna()

---
## 3️⃣ Test de France Travail API (OAuth 2.0)

**Documentation :** https://francetravail.io/data/api

**Note :** Cette API nécessite une authentification OAuth 2.0 en 2 étapes :
1. Obtenir un access token
2. Utiliser le token pour faire des requêtes

In [ ]:
def get_francetravail_token():
    """Obtient un access token OAuth 2.0 pour France Travail API"""
    
    url = "https://entreprise.francetravail.fr/connexion/oauth2/access_token"
    params = {
        'realm': '/partenaire'
    }
    data = {
        'grant_type': 'client_credentials',
        'client_id': FRANCETRAVAIL_CLIENT_ID,
        'client_secret': FRANCETRAVAIL_CLIENT_SECRET,
        'scope': 'api_offresdemploiv2 o2dsoffre'
    }
    
    try:
        response = requests.post(url, params=params, data=data, timeout=10)
        response.raise_for_status()
        
        token_data = response.json()
        return token_data.get('access_token')
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur lors de l'obtention du token : {e}")
        return None

def test_francetravail():
    """Teste la connexion à France Travail API et récupère 10 offres"""
    
    print("🔍 Test de France Travail API...")
    
    # Étape 1 : Obtenir le token
    print("  🔐 Obtention du token OAuth...")
    token = get_francetravail_token()
    
    if not token:
        print("❌ Impossible d'obtenir le token")
        return []
    
    print("  ✅ Token obtenu")
    
    # Étape 2 : Faire la requête
    url = "https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    params = {
        'motsCles': 'data',
        'range': '0-9'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        jobs = data.get('resultats', [])
        
        print(f"✅ Connexion réussie !")
        print(f"📊 Nombre d'offres récupérées : {len(jobs)}")
        
        # Afficher un exemple
        if jobs:
            print("\n📝 Exemple d'offre :")
            job = jobs[0]
            print(f"  - Titre : {job.get('intitule', 'N/A')}")
            print(f"  - Entreprise : {job.get('entreprise', {}).get('nom', 'N/A')}")
            print(f"  - Localisation : {job.get('lieuTravail', {}).get('libelle', 'N/A')}")
            print(f"  - Type contrat : {job.get('typeContrat', 'N/A')}")
            
        return jobs
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur : {e}")
        return []

# Exécuter le test
francetravail_jobs = test_francetravail()

---
## 📊 Résumé des tests

Affichons un tableau récapitulatif :

In [ ]:
import pandas as pd

# Créer un DataFrame récapitulatif
summary = pd.DataFrame({
    'API': ['The Muse', 'Adzuna', 'France Travail'],
    'Offres collectées': [len(themuse_jobs), len(adzuna_jobs), len(francetravail_jobs)],
    'Statut': [
        '✅ OK' if themuse_jobs else '❌ Échec',
        '✅ OK' if adzuna_jobs else '❌ Échec',
        '✅ OK' if francetravail_jobs else '❌ Échec'
    ]
})

print("\n" + "="*50)
print("📊 RÉSUMÉ DES TESTS")
print("="*50)
print(summary.to_string(index=False))
print("\n")

total = len(themuse_jobs) + len(adzuna_jobs) + len(francetravail_jobs)
print(f"📈 Total d'offres collectées : {total}")

if total > 0:
    print("\n🎉 Félicitations ! Les APIs fonctionnent correctement.")
    print("➡️  Prochaine étape : Créer les scripts de collecte complets")
else:
    print("\n⚠️  Aucune offre collectée. Vérifiez vos clés API.")

---
## 💾 Sauvegarder les données de test

Sauvegardons ces premières données en JSON :

In [ ]:
# Créer le dossier data/raw s'il n'existe pas
import os
os.makedirs('../data/raw', exist_ok=True)

# Sauvegarder
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

if themuse_jobs:
    with open(f'../data/raw/themuse_test_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(themuse_jobs, f, ensure_ascii=False, indent=2)
    print(f"✅ The Muse : {len(themuse_jobs)} offres sauvegardées")

if adzuna_jobs:
    with open(f'../data/raw/adzuna_test_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(adzuna_jobs, f, ensure_ascii=False, indent=2)
    print(f"✅ Adzuna : {len(adzuna_jobs)} offres sauvegardées")

if francetravail_jobs:
    with open(f'../data/raw/francetravail_test_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(francetravail_jobs, f, ensure_ascii=False, indent=2)
    print(f"✅ France Travail : {len(francetravail_jobs)} offres sauvegardées")

print(f"\n📁 Fichiers sauvegardés dans : data/raw/")

---
## ✅ Prochaines étapes

Maintenant que les APIs fonctionnent, voici ce qu'on va faire :

1. **Créer les scripts de collecte complets** (`src/collectors/`)
2. **Automatiser la collecte** de 1000+ offres
3. **Créer la base de données** PostgreSQL
4. **Développer le pipeline ETL** pour nettoyer les données

**Rendez-vous dans le prochain notebook !** 🚀